# Python in Comparison with Other Languages

Someone that is familiar with arrays in the C language could write Python code similar to the following example:


In [1]:
some_list = []
for index in range(len(some_list)):
    print(some_list[index])


An experienced Pythonic programmer would most probably write:


In [2]:
for item in some_list:
    print(item)

## Accessing super-classes

Consider a subclass of Python's dictionary type, which allows access to the stored keys through a case-insensitive key lookup. 
The following is an example of such an implementation:


In [3]:
!python _01_Accessing_super_classes/caseinsensitive.py

FOO: bar
foo: bar
biz: baz
BIZ: baz


Good practices when subclassing built-in types:
* Trying to directly subclass the built-in types 'dict' and 'list' can lead to subtle bugs.
* For this reason, use colections.UserDict and collections.UserList for subclassing the dict and list types, respectively.


## Multiple inheritance and Method Resolution Order

The MRO algorithm in python is based on the 'C3 linearization' algorithm. It's useful in 'diamond class hierarchy' scenarios, such as the following example:


In [4]:
!python _02_Multiple_iheritance_MRO/mro.py

("MyClass's MRO: \n"
 "(<class '__main__.MyClass'>, <class '__main__.Base1'>, <class "
 "'__main__.Base2'>, <class '__main__.CommonBase'>, <class 'object'>)")


## Class instance initialization

Best practices when initializing classes:
* Python classes do not require you to define attributes in the class body.
* Defining attributes in the class body is also **dangerous**: it can lead to very subtle bugs if one decides to assign as a class attribute a mutable type like list or dict.


To see the issues with mutable class attributes in action, consider the following class with mutable types as class attributes:

In [5]:
from _03_Class_instance_init.aggregator_shared import AggregatorShared


Let's create 2 instances of class 'AggregatorShared' and start adding elements to the 2 instances:

In [6]:
a1 = AggregatorShared()
a2 = AggregatorShared()
a1.aggregate("a1-1")
a1.aggregate("a1-2")
a2.aggregate("a2-1")


When checking the 'all_aggregated' attribute of both instances, they are the same!. It's like they're not storing their own values in 'all_aggregated', but sharing all values added in a single list:

In [7]:
a1.all_aggregated

['a1-1', 'a1-2', 'a2-1']

In [8]:
a2.all_aggregated

['a1-1', 'a1-2', 'a2-1']

When checking the 'last_aggregated' attribute of both instances, they are the last values added in each instance, as expected:

In [9]:
a1.last_aggregated


'a1-2'

In [10]:
a2.last_aggregated


'a2-1'

By inspecting the class attribute values, we see that all 'AggregatorShared' instances shared their state
through the mutable 'all_aggregated' attribute:

In [11]:
AggregatorShared.all_aggregated


['a1-1', 'a1-2', 'a2-1']

In [12]:
AggregatorShared.last_aggregated


To avoid the last issue, all attribute values that are supposed to be unique for every class instance should be initialized in the `__init__()` method only.

Now let's create 2 instances of class 'AggregatorIndependent' and start adding elements to the 2 instances:


In [13]:
from _03_Class_instance_init.aggregator_independent import AggregatorIndependent


a1 = AggregatorIndependent()
a2 = AggregatorIndependent()
a1.aggregate("a1-1")
a1.aggregate("a1-2")
a2.aggregate("a2-1")


Now all instance objects attributes have their own values in each attribute, as expected:

In [14]:
a1.all_aggregated


['a1-1', 'a1-2']

In [15]:
a2.all_aggregated

['a2-1']

In [16]:
a1.last_aggregated

'a1-2'

In [17]:
a2.last_aggregated

'a2-1'

## Attribute access patterns

Python lacks the notion of public, private, and protected class attributes found in other OOP languages:

* Private attributes restrict access to specific symbols from anyone outside of a specific class.

* Protected attributes restrict access to specific symbols from anyone outside of the inheritance tree.


The only similar thing Python has is 'name mangling': if an attribute is prefixed by __ (two underscores) within a class body, it's renamed by the interpreter:

In [18]:
class MyClass:
    def __init__(self):
        self.__secret_value = 1


instance_of = MyClass()


In [19]:
instance_of.__secret_value


AttributeError: 'MyClass' object has no attribute '__secret_value'

In [20]:
instance_of._MyClass__secret_value

1

In Python:

* Name mangling does not restrict attribute access, it only makes it less convenient.

* But it can help avoid naming collisions in the inheritance tree. Still, it's not recommended to use name mangling in base classes by default, to avoid any collisions in advance.


## Descriptors

The descriptor classes are based on 3 special methods that form the descriptor protocol:

* `__set__`(self, obj, value): This is called whenever the attribute is set. (a.k.a. setter)

* `__get__`(self, obj, owner=None): This is called whenever the attribute is read. (a.k.a. getter)

* `__delete__`(self, obj): This is called when del is invoked on the attribute.

* A descriptor that implements `__get__()` and `__set__()` is called a data descriptor.

* If it just implements `__get__()`, then it is called a non-data descriptor.


Consider the following example that shows how descriptors work: 

In [21]:
!python _04_Descriptors/reveal_access.py


Retrieving var "x"
10
Updating var "x"
Retrieving var "x"
20
5
Deleting var "x"


The preceding example shows that:

* The `__get__()` method is called whenever the instance attribute is retrieved. 

* The `__set__()` method is called whenever a value is assigned to the instance attribute. 

* The `__del__()` method is called whenever an instance attribute is deleted.

* Descriptors, in order to work, need to be defined as class attributes. 


## Lazily evaluated attributes

As an example usage of descriptors, consider a class 'WithSortedRandoms' where all instances have access to a shared list of sorted random values: 

* The length of the list can be arbitrarily long, so it makes sense to sort it once and reuse it for all instances. 

* So the list will be initialized only on first access. 

Here is an example usage of the 'WithSortedRandoms' class:


In [22]:
!python _05_lazily_evaluated_attributes/lazily_evaluated.py


initialized!
[0.05187589683610638, 0.20347074966617795, 0.335194338415429, 0.8931927622289663, 0.9584916674144855]
cached!
[0.05187589683610638, 0.20347074966617795, 0.335194338415429, 0.8931927622289663, 0.9584916674144855]


Also, a data descriptor can be implemented to be used as a decorator, as follows:


In [23]:
!python _05_lazily_evaluated_attributes/lazy_property.py


[[0.8182933774323621, 0.4278356278017691, 0.7359357327077695, 0.3443619169080848, 0.95180158266743]]
[[0.8182933774323621, 0.4278356278017691, 0.7359357327077695, 0.3443619169080848, 0.95180158266743]]


This is useful when:
* An object instance needs to be stored as a class attribute that's shared between its instances (to save resources).
* The object can't be initialized at import time because some global application state/context is needed.


## Properties


In other languages:

* It's common to have private methods by default and a getter an setter for every attribute.

* But with a large number of attributes, this means tons of getters and setter methods, which obscures the intent of the class.
 

In [24]:
class UserAccount:
    def __init__(self, username, password):
        self._username = username
        self._password = password

    def get_username(self):
        return self._username

    def set_username(self, username):
        self._username = username

    def get_password(self):
        return self._password

    def set_username(self, password):
        self._password = password


With python's properties:

* You can expose class attributes as public by default.

* Then convert one attribute to private and add getter and setter methods when necessary.


In [25]:
class UserAccount:
    def __init__(self, username, password):
        self.username = username
        self._password = password

    @property
    def password(self):
        return self._password
    
    @password.setter
    def password(self, value):
        self._password = value


## Dynamic polymorphism

Python's mechanism of polymorphism is called "duck typing": 

* This means that any object can be used within a given context as long as the object works and behaves as the context expects.

* "If it walks like a duck and it quacks like a duck, then it must be a duck".


Consider the following example:

In [26]:
def printfile(file):
    try:
        contents = file.read()
        print(file)
    finally:
        file.close()


This function won't raise any exception as long as:

* The file argument has a read() method.

* The result of file.read() is a valid argument to the print() function.

* The file argument has the close() method.


## Dunder methods (language protocols)

To see dunder methods in action, consider the following implementation of matrix operations (+, -, *):

In [27]:
!python _06_Dunder_methods/matrices.py


Matrix m1: 
[1, 2, 3]
[4, 1, 4]
[5, 7, 9]

Matrix m2: 
[1, 2, 3]
[1, 4, 3]
[1, 0, 5]

Sum of m1 and m2: 
[2, 4, 6]
[5, 5, 7]
[6, 7, 14]

Substraction of m1 and m2: 
[0, 0, 0]
[3, -3, 1]
[4, 7, 4]

Multiplication of m1 and m2: 
[6, 10, 24]
[9, 12, 35]
[21, 38, 81]



We can add support for multiplication between a matrix and a scalar as follows:

In [1]:
!python _06_Dunder_methods/matrices_with_scalars.py


Matrix m1: 
[1, 2, 3]
[4, 1, 4]
[5, 7, 9]

Matrix m2: 
[1, 2, 3]
[1, 4, 3]
[1, 0, 5]

Multiplication of m1 by 2: 
[2, 4, 6]
[8, 2, 8]
[10, 14, 18]

Multiplication of m2 by 3: 
[3, 6, 9]
[3, 12, 9]
[3, 0, 15]

